In [31]:
from Bio.Seq import Seq, MutableSeq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import random
import subprocess
import os

### Mutate gene and check for missense

In [13]:
def point_mutate(gene):
    new_gene = MutableSeq(gene)
    pos = random.randint(0, len(gene)-1)
    old_base = gene[pos]
    base_choices = ['A', 'T', 'C', 'G']
    base_choices.remove(old_base)
    new_base = random.choice(base_choices)
    new_gene[pos] = new_base
    return Seq(new_gene)

In [15]:
seq_record = SeqIO.read('data/lacz_gene.fna', 'fasta')
gene = seq_record.seq

In [17]:
new_gene = point_mutate(gene)
protein = gene.translate()
new_protein = new_gene.translate()
[i for i in range(len(protein)) if protein[i] != new_protein[i]]

[904]

### Save proteins for original and mutated gene in fasta format

In [19]:
orig_record = SeqRecord(
    protein,
    id='original_lacz_protein',
    description='original lacz protein',
)

mutated_record = SeqRecord(
    new_protein,
    id='mutated_lacz_protein',
    description='mutated lacz protein',
)

SeqIO.write(orig_record, 'data/original_lacz_protein.fasta', 'fasta')
SeqIO.write(mutated_record, 'data/mutated_lacz_protein.fasta', 'fasta')

1

### Run LocalColabFold

In [36]:
def fold(input_file, output_dir):
    command = [
        "colabfold_batch", 
        input_file,
        output_dir,
        ## command args here
    ]

    custom_env = os.environ.copy()
    custom_env["PATH"] = '/usr/local/cuda-12.6/bin:/home/ben/gems/bin:/home/ben/.pyenv/shims:/home/ben/code/localcolabfold/colabfold-conda/bin:/home/ben/code/localcolabfold/colabfold-conda/bin:/usr/local/cuda-12.6/bin:/home/ben/gems/bin:/usr/local/cuda-12.6/bin:/home/ben/gems/bin:/usr/local/cuda-12.6/bin:/home/ben/gems/bin:/home/ben/gems/bin:/home/ben/.nvm/versions/node/v21.5.0/bin:/home/ben/.pyenv/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin'

    try:
        result = subprocess.run(
            command,
            capture_output=True,
            text=True,
            check=True,
            env=custom_env,
        )
        print("Command executed successfully.")
        print("Output:", result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error during execution:", e.stderr)

In [37]:
fold('data/original_lacz_protein.fasta', 'original_lacz_protein_output/')

Error during execution: WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
E0000 00:00:1731537279.722701   60618 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731537279.728185   60618 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

limited shared resource only capable of processing a few thousand MSAs per day. Please
submit jobs only from a single IP address. We reserve the right to limit access to the
server case-by-case when usage exceeds fair use. If you require more MSAs: You can 
precompute all MSAs with `colabfold_search` or host your own API and pass it to `--host-url`

Traceback (most recent call last):
  File "/home/ben/code/localcolabfold/colabfold-conda/bin/colabfold_batch", line 8, in <module>
    sys.exit(main())
  File "/home/ben/code/localcol